<a href="https://colab.research.google.com/github/fwang312/Airbnb-User-Booking/blob/master/New_User_Booking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DA 1912 New User Booking Project

 Airbnb​​ ​ is​ ​ an​ ​ online​ ​ marketplace​ ​ and​ ​ hospitality​ ​ service,​ ​ enabling​ ​ people​ ​ to​ ​ lease​ ​ or​ ​ rent​ ​ short-term
lodging​ ​ including​ ​ vacation​ ​ rentals​ ​ , ​ ​ apartment​ ​ rentals,​ ​ homestays​ ​ , ​ ​ hostels​ ​ beds,​ ​ or​ ​ hotel​ ​ roo​ ms.
New​ ​ users​ ​ on​ ​ Airbnb​ ​ can​ ​ book​ ​ a ​ ​ place​ ​ to​ ​ stay​ ​ in​ ​ 34,000+​ ​ cities​ ​ across​ ​ 190+​ ​ countries.​ ​ 

- **Objective :**

   To predict travel destinations of users
   
-  **Dataset** -- Train_users.csv
 
    There are 16 features used to describe each user in the dataset:
    *         user id
    *         the date of account creation
    *         timestamp of the first activity, note that it can be earlier than
    *         date of first booking
    *         gender
    *         age
    *         signup_method
    *         the page a user came to signup up from
    *         international language preference
    *         what kind of paid marketing
    *         where the marketing is e.g. google, craigslist, other
    *         whats the first marketing the user interacted with before the signing up
    *         signup_app
    *         first_device_type
    *         first_browser